In [54]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [55]:
# Load the training dataset
train_data = pd.read_csv('data/train.csv')

# Load the test dataset
test_data = pd.read_csv('data/test.csv')


In [56]:
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler

# Step 1: Handle duplicates and invalid dates
train_data = train_data.drop_duplicates(subset=['listing_id'])
current_year = datetime.now().year

# Convert reg_date once and store
train_data['reg_date'] = pd.to_datetime(train_data['reg_date'], format='mixed', dayfirst=True)

# Filter invalid dates and years
train_data = train_data[
    (train_data['manufactured'] <= current_year) & 
    (train_data['manufactured'] >= 1960) &  # Remove unrealistic old cars
    (train_data['reg_date'] <= datetime.now())
]

# Step 2: Remove uninformative columns
train_data = train_data.drop(columns=[
    'listing_id', 'title', 'description', 'features', 'accessories'
])

# Remove columns with single unique value
train_data = train_data.drop(columns=train_data.columns[train_data.nunique() == 1])

# Step 3: Handle missing values strategically
# Remove columns with excessive missing values
na_threshold = 0.5
na_columns = train_data.columns[train_data.isnull().mean() > na_threshold]
train_data = train_data.drop(columns=na_columns)

# Step 4: Feature engineering
# Calculate registration age
train_data['coe_days'] = (datetime.now() - train_data['reg_date']).dt.days
train_data['vehicle_age'] = current_year - train_data['manufactured']
train_data = train_data.drop(columns=['reg_date'])

# Step 5: Handle categorical variables
if 'make' in train_data.columns:
    # Handle make categories
    train_data['make'] = train_data['make'].fillna('Unknown')
    make_counts = train_data['make'].value_counts()
    min_count_threshold = max(10, len(train_data) * 0.01)  # Dynamic threshold
    
    def replace_make(x):
        if x == 'Unknown':
            return 'Unknown'
        return 'Other' if make_counts.get(x, 0) < min_count_threshold else x
    
    train_data['make'] = train_data['make'].apply(replace_make)

# Step 6: Handle categorical columns with careful ordering
categorical_cols = ['make', 'model', 'type_of_vehicle', 'transmission', 'fuel_type', 'eco_category']
existing_categorical_cols = [col for col in categorical_cols if col in train_data.columns]

# Convert categorical columns to 'category' type
for col in existing_categorical_cols:
    train_data[col] = train_data[col].astype('category')

# One-hot encoding with handling of unknown categories
train_data = pd.get_dummies(
    train_data, 
    columns=existing_categorical_cols, 
    dummy_na=True,  # Handle unknown categories in test data
    drop_first=True
)

# Capture column names to use as a reference for test_data
train_columns = train_data.columns

# Step 7: Advanced numeric feature handling
numeric_features = ['curb_weight', 'power', 'engine_cap', 'mileage', 'omv', 
                   'arf', 'road_tax', 'coe', 'dereg_value']
numeric_features = [col for col in numeric_features if col in train_data.columns]

# Two-step imputation
# First, fill extreme outliers with median
for col in numeric_features:
    if train_data[col].dtype in ['int64', 'float64']:
        Q1 = train_data[col].quantile(0.25)
        Q3 = train_data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        train_data[col] = train_data[col].clip(lower_bound, upper_bound)

# Then apply SimpleImputer
imputer = SimpleImputer(strategy='median')
train_data[numeric_features] = imputer.fit_transform(train_data[numeric_features])

# Step 8: Scaling
numeric_cols = train_data.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = numeric_cols.drop('price', errors='ignore')

# Use RobustScaler for better handling of outliers
scaler = RobustScaler()
train_data[numeric_cols] = scaler.fit_transform(train_data[numeric_cols])

train_data.head()


,manufactured,category,curb_weight,power,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,...,type_of_vehicle_mpv,type_of_vehicle_others,type_of_vehicle_sports car,type_of_vehicle_stationwagon,type_of_vehicle_suv,type_of_vehicle_truck,type_of_vehicle_van,type_of_vehicle_nan,transmission_manual,transmission_nan
0,0.2,parf car,1.022785,3.072727,2.381710,0.0,2.506122,0.130314,1.657552,2.247846,...,False,False,False,False,True,False,False,False,False,False
1,0.0,"parf car, premium ad car",-0.037975,0.436364,0.385686,0.0,0.723810,0.082009,0.123698,0.490008,...,False,False,False,False,False,False,False,False,False,False
2,-2.0,"coe car, premium ad car, low mileage car",0.425316,0.127273,1.107356,0.5,-0.453061,0.242532,1.738281,-0.392055,...,True,False,False,False,False,False,False,False,False,False
3,-1.8,"coe car, premium ad car",-0.721519,-0.563636,-0.395626,0.5,-0.776871,-0.848238,0.007812,-0.507513,...,False,False,False,False,False,False,False,False,False,False
4,-2.2,"coe car, premium ad car",0.455696,1.309091,2.381710,2.0,-0.293878,0.152719,3.207031,-0.599601,...,False,False,False,False,False,False,False,False,False,False


In [57]:
target = 'price'

In [58]:
X = train_data
y = train_data[target]

In [59]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# Separate features and target variable from training data
X = train_data.drop(columns=['price'])  # Replace 'price' with the actual target column name if different
y = train_data['price']  # Replace 'price' with the actual target column name if different

# Step 1: Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = pd.get_dummies(X_train, drop_first=True)
X_valid = pd.get_dummies(X_valid, drop_first=True)

# Align columns to ensure compatibility
X_train, X_valid = X_train.align(X_valid, join='left', axis=1, fill_value=0)


# Step 4: Create preprocessing pipelines for both numeric and categorical features
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# Step 5: Combine preprocessing steps into a column transformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, train)
#     ]
# )

# Step 6: Create a pipeline that first preprocesses the data and then applies the XGBoost model
model_pipeline = Pipeline(steps=[
    ('model', XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=20, random_state=42))
])

# Step 3: Fit the model pipeline to the training data

print(X_train)
model_pipeline.fit(X_train, y_train)

# Step 4: Make predictions on the validation set
y_pred = model_pipeline.predict(X_valid)

# Step 5: Evaluate the model's performance
mae = mean_absolute_error(y_valid, y_pred)
mse = mean_squared_error(y_valid, y_pred)
rmse = mean_squared_error(y_valid, y_pred, squared=False)
r2 = r2_score(y_valid, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
print(f'R-squared (R2): {r2:.2f}')


       manufactured  curb_weight     power  engine_cap  no_of_owners  \
14465           1.0     1.627848  1.618182    0.393638          -0.5   
2714           -1.0     0.086076  4.345455    3.978131          -0.5   
11614          -2.0     0.759494  0.000000    2.355865           1.5   
13495          -1.6    -0.579747 -0.345455   -0.409543           2.0   
7796           -0.6     0.810127  0.000000    1.373757           0.0   
...             ...          ...       ...         ...           ...   
21589           0.8    -0.215190 -0.018182   -0.594433          -0.5   
5396           -1.2     1.101266  2.254545    2.349901           2.0   
861             0.4    -0.886076 -0.290909   -0.598410           0.0   
15807           0.0    -0.632911 -0.272727   -0.395626          -0.5   
23672          -0.2     0.367089  0.218182    1.119284           0.0   

       depreciation       coe  road_tax  dereg_value   mileage  ...  \
14465      1.497959  3.481042  0.131510     3.288237 -1.591688  

H:\bin\envs\sem4_assignments\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [60]:
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler

# Step 1: Handle duplicates and invalid dates
test_data = test_data.drop_duplicates(subset=['listing_id'])
current_year = datetime.now().year

# Convert reg_date once and store
test_data['reg_date'] = pd.to_datetime(test_data['reg_date'], format='mixed', dayfirst=True)

# Filter invalid dates and years
test_data = test_data[
    (test_data['manufactured'] <= current_year) & 
    (test_data['manufactured'] >= 1960) &  # Remove unrealistic old cars
    (test_data['reg_date'] <= datetime.now())
]

# Step 2: Remove uninformative columns
test_data = test_data.drop(columns=[
    'listing_id', 'title', 'description', 'features', 'accessories'
])

# Remove columns with single unique value
test_data = test_data.drop(columns=test_data.columns[test_data.nunique() == 1])

# Step 3: Handle missing values strategically
# Remove columns with excessive missing values
na_threshold = 0.5
na_columns = test_data.columns[test_data.isnull().mean() > na_threshold]
test_data = test_data.drop(columns=na_columns)

# Step 4: Feature engineering
# Calculate registration age
test_data['coe_days'] = (datetime.now() - test_data['reg_date']).dt.days
test_data['vehicle_age'] = current_year - test_data['manufactured']
test_data = test_data.drop(columns=['reg_date'])

# Step 5: Handle categorical variables
if 'make' in test_data.columns:
    # Handle make categories
    test_data['make'] = test_data['make'].fillna('Unknown')
    make_counts = test_data['make'].value_counts()
    min_count_threshold = max(10, len(test_data) * 0.01)  # Dynamic threshold
    
    def replace_make(x):
        if x == 'Unknown':
            return 'Unknown'
        return 'Other' if make_counts.get(x, 0) < min_count_threshold else x
    
    test_data['make'] = test_data['make'].apply(replace_make)

# Step 6: Handle categorical columns with careful ordering
categorical_cols = ['make', 'model', 'type_of_vehicle', 'transmission', 'fuel_type', 'eco_category']
existing_categorical_cols = [col for col in categorical_cols if col in test_data.columns]

# Convert categorical columns to 'category' type
for col in existing_categorical_cols:
    test_data[col] = test_data[col].astype('category')

# One-hot encoding with handling of unknown categories
test_data = pd.get_dummies(
    test_data, 
    columns=existing_categorical_cols, 
    dummy_na=True,  # Handle unknown categories in test data
    drop_first=True
)

# Align the test set to have the same columns as the train set
# test_data = test_data.reindex(columns=train_columns, fill_value=0)

# Step 7: Advanced numeric feature handling
numeric_features = ['curb_weight', 'power', 'engine_cap', 'mileage', 'omv', 
                   'arf', 'road_tax', 'coe', 'dereg_value']
numeric_features = [col for col in numeric_features if col in test_data.columns]

# Two-step imputation
# First, fill extreme outliers with median
for col in numeric_features:
    if test_data[col].dtype in ['int64', 'float64']:
        Q1 = test_data[col].quantile(0.25)
        Q3 = test_data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        test_data[col] = test_data[col].clip(lower_bound, upper_bound)

# Then apply SimpleImputer
imputer = SimpleImputer(strategy='median')
test_data[numeric_features] = imputer.fit_transform(test_data[numeric_features])

# Step 8: Scaling
numeric_cols = test_data.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = numeric_cols.drop('price', errors='ignore')

# Use RobustScaler for better handling of outliers
scaler = RobustScaler()
test_data[numeric_cols] = scaler.fit_transform(test_data[numeric_cols])

test_data.head()


,manufactured,category,curb_weight,power,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,...,type_of_vehicle_mpv,type_of_vehicle_others,type_of_vehicle_sports car,type_of_vehicle_stationwagon,type_of_vehicle_suv,type_of_vehicle_truck,type_of_vehicle_van,type_of_vehicle_nan,transmission_manual,transmission_nan
0,-0.4,parf car,-0.743590,-0.296296,-0.456233,0.0,0.242301,0.573655,-0.591221,-0.569939,...,False,False,False,False,True,False,False,False,False,False
1,-2.0,"coe car, premium ad car, low mileage car",-0.628205,-0.611111,-0.301629,-0.5,-0.670051,-0.126082,0.000000,-0.447729,...,False,False,False,False,False,False,False,False,False,False
2,0.4,parf car,0.141026,0.537037,0.304661,-0.5,0.846024,-0.592876,0.133059,0.791043,...,False,False,True,False,False,False,False,False,False,False
3,0.4,"parf car, premium ad car",-0.974359,-0.611111,-0.456233,0.5,-0.293739,-0.767009,-0.591221,-0.065062,...,False,False,False,False,False,False,False,False,False,False
4,-0.4,"parf car, premium ad car",-0.400000,-0.444444,-0.456233,0.5,-0.227411,0.516376,-0.591221,-0.401005,...,False,False,False,False,False,False,False,False,False,False


In [61]:
train_data_columns = train_data.columns

X_test = pd.get_dummies(test_data, drop_first=True)

X_test = X_test[train_data_columns]
X_test = X_test.reindex(columns=train_data_columns, fill_value=0)
X_test.drop(columns=['price'], inplace=True)
test_data_columns = X_test.columns

# Find columns in train_data that are not in test_data
missing_columns = train_data_columns.difference(test_data_columns)

print(missing_columns)

KeyError: "['category', 'price', 'make_mini', 'model_107', 'model_120y', 'model_159', 'model_190', 'model_1m', 'model_2000', 'model_200zr', 'model_216', 'model_218d', 'model_220', 'model_230.6', 'model_230e', 'model_280zx', 'model_300ce', 'model_318ci', 'model_323ci', 'model_323i', 'model_328', 'model_330ci', 'model_340i', 'model_340r', 'model_4c', 'model_504', 'model_635', 'model_730ld', 'model_750i', 'model_850', 'model_850m', 'model_914', 'model_928', 'model_944', 'model_981', 'model_actros', 'model_airtrek', 'model_alfetta', 'model_altezza', 'model_atlas', 'model_auris', 'model_axor', 'model_b9l', 'model_barracuda', 'model_bj1041', 'model_bj611', 'model_c40', 'model_c70', 'model_capri', 'model_cla45', 'model_clio', 'model_copen', 'model_corniche', 'model_corona', 'model_cr-z', 'model_cyh52t', 'model_cyz52l', 'model_delica', 'model_dino', 'model_ds5', 'model_e-2008', 'model_e-c4', 'model_e-deliver', 'model_e-up', 'model_e240', 'model_e500', 'model_ec31', 'model_eclat', 'model_elan',

In [50]:


test_predictions = model_pipeline.predict(X_test)

output = pd.DataFrame({'Id': X_test.index, 'Predicted': test_predictions})
output.to_csv('predictions_v1.1.csv', index=False)

print(output.head())

ValueError: feature_names mismatch: ['manufactured', 'curb_weight', 'power', 'engine_cap', 'no_of_owners', 'depreciation', 'coe', 'road_tax', 'dereg_value', 'mileage', 'omv', 'arf', 'coe_days', 'vehicle_age', 'make_Unknown', 'make_audi', 'make_bmw', 'make_honda', 'make_hyundai', 'make_kia', 'make_lexus', 'make_mazda', 'make_mercedes-benz', 'make_mini', 'make_mitsubishi', 'make_nissan', 'make_porsche', 'make_subaru', 'make_toyota', 'make_volkswagen', 'make_volvo', 'make_nan', 'model_107', 'model_116d', 'model_116i', 'model_118i', 'model_120i', 'model_120y', 'model_125i', 'model_12c', 'model_135i', 'model_159', 'model_1750', 'model_190', 'model_1m', 'model_2', 'model_200', 'model_2000', 'model_2002', 'model_2008', 'model_200sx', 'model_200zr', 'model_208', 'model_216', 'model_216d', 'model_216i', 'model_218d', 'model_218i', 'model_220', 'model_220i', 'model_225xe', 'model_228i', 'model_230.6', 'model_230ce', 'model_230e', 'model_230i', 'model_230sl', 'model_250', 'model_280', 'model_280z

In [ ]:
# # Display the data types of each column
# print(train_data.dtypes)

# # Check for columns with missing values and count them
# missing_values = train_data.isnull().sum()
# print(missing_values)

# # To display only columns with missing values:
# missing_columns = missing_values[missing_values > 0]
# print("Columns with missing values and their counts:")
# print(missing_columns)


In [ ]:
# # Display the first few rows of the training data
# # print(train_data.head())

# # Check for any missing values in the training data
# print(train_data.isnull().sum())


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# import numpy as np
# import joblib

# # Assume you have X_train and X_valid as your training and validation datasets
# # Identify categorical and numerical features
# # categorical_features = X_train.select_dtypes(include=['object']).columns
# # numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns

# # Preprocessing pipeline for categorical features (OneHotEncoder in this example)
# # categorical_transformer = Pipeline(steps=[
# #     ('imputer', SimpleImputer(strategy='most_frequent')),
# #     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# # ])

# # Preprocessing pipeline for numerical features
# # numerical_transformer = Pipeline(steps=[
# #     ('imputer', SimpleImputer(strategy='mean'))
# # ])

# # Combine preprocessing steps
# # preprocessor = ColumnTransformer(
# #     transformers=[
# #         ('num', numerical_transformer, numerical_features),
# #         ('cat', categorical_transformer, categorical_features)
# #     ]
# # )

# # Create the complete pipeline including preprocessing and the XGBRegressor model
# pipeline = Pipeline(steps=[
#     ('model', XGBRegressor(random_state=42))
# ])

# # Define the parameter grid to search over
# param_grid = {
#     'model__n_estimators': [500],
#     'model__learning_rate': [0.1],
#     'model__max_depth': [20],
#     'model__subsample': [0.5, 0.6, 0.8, 1.0],
#     'model__colsample_bytree': [0.4, 0.6, 0.8, 1.0],
#     'model__min_child_weight': [1, 5, 10, 20]
# }

# # Set up RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=pipeline,
#     param_distributions=param_grid,
#     n_iter=10,  # Number of different combinations to try
#     scoring='neg_mean_absolute_error',  # Use MAE as the scoring metric
#     cv=5,  # 5-fold cross-validation
#     verbose=2,
#     random_state=42,
#     n_jobs=1  # Use all available cores
# )

# # Fit the RandomizedSearchCV on the training data
# random_search.fit(X_train, y_train)

# # Get the best model and parameters
# best_model = random_search.best_estimator_
# best_params = random_search.best_params_

# # Save the best model to a file
# joblib.dump(best_model, 'best_xgb_model.pkl')

# # Evaluate the model with the best parameters on validation data
# y_pred = best_model.predict(X_valid)
# mae = mean_absolute_error(y_valid, y_pred)
# mse = mean_squared_error(y_valid, y_pred)
# rmse = np.sqrt(mse)

# print(f'Best Parameters: {best_params}')
# print(f'MAE: {mae}')
# print(f'MSE: {mse}')
# print(f'RMSE: {rmse}')
# #



In [240]:
# import pandas as pd
# import numpy as np

# # Example DataFrame (replace with your actual data)
# # train_data = pd.read_csv('your_dataset.csv')

# # Step 1: Check for NaN or infinite values in 'manufactured'
# nan_count = train_data['manufactured'].isna().sum()
# infinite_count = (~np.isfinite(train_data['manufactured'])).sum()

# print(nan_count, "NaN values in manufactured")
# print(infinite_count, "Infinite values in manufactured")

# # Step 2: Handle NaN and infinite values
# # Replace infinite values with NaN and avoid chained assignment
# train_data['manufactured'] = train_data['manufactured'].replace([np.inf, -np.inf], np.nan)

# # Fill NaN with 0 (or another value you deem appropriate)
# train_data['manufactured'] = train_data['manufactured'].fillna(0)

# # Step 3: Convert 'manufactured' to int
# train_data['manufactured'] = train_data['manufactured'].astype(int)

# # Convert 'original_reg_date' and 'reg_date' to datetime
# # Specify format if known, otherwise keep using errors='coerce'
# train_data['original_reg_date'] = pd.to_datetime(train_data['original_reg_date'], errors='coerce', format='%Y-%m-%d')  # Adjust format as needed
# train_data['reg_date'] = pd.to_datetime(train_data['reg_date'], errors='coerce', format='%Y-%m-%d')  # Adjust format as needed

# # Convert categorical columns to 'category' dtype for efficiency
# categorical_columns = [
#     'make', 'model', 'type_of_vehicle', 'category', 'transmission', 
#     'fuel_type', 'opc_scheme', 'lifespan', 'eco_category', 
#     'features', 'accessories'
# ]

# for column in categorical_columns:
#     train_data[column] = train_data[column].astype('category')

# # Convert 'no_of_owners' and 'depreciation' to integers if applicable
# train_data['no_of_owners'] = train_data['no_of_owners'].fillna(0).astype(int)  # Filling NaN with 0
# train_data['depreciation'] = train_data['depreciation'].fillna(0).astype(int)  # Filling NaN with 0

# # Display the data types to verify changes
# print(train_data.dtypes) 


In [ ]:
# import pandas as pd
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from sklearn.model_selection import GridSearchCV

# # Step 1: Define numerical and categorical features
# numerical_features = [
#     'manufactured', 'curb_weight', 'power', 'engine_cap', 
#     'no_of_owners', 'depreciation', 'coe', 'road_tax', 
#     'dereg_value', 'mileage', 'omv', 'arf'
# ]
# categorical_features = [
#     'make', 'model', 'type_of_vehicle', 'category', 
#     'transmission', 'fuel_type', 'opc_scheme', 
#     'lifespan', 'eco_category', 'features', 'accessories'
# ]
# # Separate features and target variable from training data
# X = train_data.drop(columns=['price'])  # Replace 'target' with the actual target column name
# y = train_data['price']  # Replace 'target' with the actual target column name

# # Separate features from test data
# X_test = test_data  # Assuming test data has the same feature columns as training data


# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# # Step 3: Create preprocessing pipelines for both numeric and categorical features
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Step 4: Combine preprocessing steps into a column transformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_features),
#         ('cat', categorical_transformer, categorical_features)
#     ]
# )

# # Step 5: Create a pipeline that first preprocesses the data and then applies the XGBoost model
# model_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('model', XGBRegressor(random_state=42))  # Temporarily remove n_estimators, learning_rate, max_depth
# ])

# # Define a parameter grid for hyperparameter tuning
# param_grid = {
#     'model__n_estimators': [100, 200, 300],
#     'model__learning_rate': [0.01, 0.1, 0.2],
#     'model__max_depth': [3, 5, 7, 10],
#     'model__subsample': [0.6, 0.8, 1.0]
# }

# # Use GridSearchCV to search for the best parameters
# grid_search = GridSearchCV(model_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
# grid_search.fit(X_train, y_train)

# # Get the best model from grid search
# best_model = grid_search.best_estimator_

# # Make predictions with the best model
# y_pred = best_model.predict(X_valid)

# # Evaluate the model's performance
# mae = mean_absolute_error(y_valid, y_pred)
# mse = mean_squared_error(y_valid, y_pred)
# rmse = mse ** 0.5  # or use np.sqrt(mse) 
# r2 = r2_score(y_valid, y_pred)

# # Print the evaluation metrics
# print(f'Mean Absolute Error (MAE): {mae:.2f}')
# print(f'Mean Squared Error (MSE): {mse:.2f}')
# print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
# print(f'R-squared (R2): {r2:.2f}')


In [57]:
# # Separate features and target variable from training data
# X = train_data.drop(columns=['price'])  # Replace 'target' with the actual target column name
# y = train_data['price']  # Replace 'target' with the actual target column name

# # Separate features from test data
# X_test = test_data  # Assuming test data has the same feature columns as training data


In [ ]:
# import pandas as pd
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# # Step 1: Define numerical and categorical features
# numerical_features = [
#     'manufactured', 'curb_weight', 'power', 'engine_cap', 
#     'no_of_owners', 'depreciation', 'coe', 'road_tax', 
#     'dereg_value', 'mileage', 'omv', 'arf'
# ]
# categorical_features = [
#     'make', 'model', 'type_of_vehicle', 'category', 
#     'transmission', 
#     'lifespan', 'features'
# ]
# # Separate features and target variable from training data
# X = train_data.drop(columns=['price'])  # Replace 'target' with the actual target column name
# y = train_data['price']  # Replace 'target' with the actual target column name

# # Separate features from test data
# X_test = test_data  # Assuming test data has the same feature columns as training data


# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# # Step 3: Create preprocessing pipelines for both numeric and categorical features
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Step 4: Combine preprocessing steps into a column transformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_features),
#         ('cat', categorical_transformer, categorical_features)
#     ]
# )

# # Step 5: Create a pipeline that first preprocesses the data and then applies the XGBoost model
# model_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     # ('model', RandomForestRegressor(n_estimators=100, random_state=42))
#     ('model', XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=20, random_state=42))
# ])

# # Step 6: Split the data into training and testing sets


# # Step 7: Fit the model pipeline to the training data
# model_pipeline.fit(X_train, y_train)

# # Step 8: Make predictions on the test set
# y_pred = model_pipeline.predict(X_valid)

# # Step 9: Evaluate the model's performance
# mae = mean_absolute_error(y_valid, y_pred)
# mse = mean_squared_error(y_valid, y_pred)
# rmse = mean_squared_error(y_valid, y_pred, squared=False)
# r2 = r2_score(y_valid, y_pred)

# # Print the evaluation metrics
# print(f'Mean Absolute Error (MAE): {mae:.2f}')
# print(f'Mean Squared Error (MSE): {mse:.2f}')
# print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
# print(f'R-squared (R2): {r2:.2f}')

In [ ]:
# import pandas as pd

# # Load your DataFrame (assuming it's already loaded as df)

# # Step 1: Convert categorical columns to category dtype
# # categorical_columns = [
# #     'make', 'model', 'type_of_vehicle', 'category', 'transmission', 
# #     'fuel_type', 'opc_scheme', 'lifespan', 'eco_category', 
# #     'features', 'accessories'
# # ]

# # for col in categorical_columns:
# #     train_data[col] = train_data[col].astype('category')

# # Step 2: Handle object columns
# # Option 1: Drop them if not useful
# train_data.drop(columns=['title', 'description'], inplace=True)

# # Option 2: Encode if they are meaningful (Example: using one-hot encoding)
# # df = pd.get_dummies(df, columns=['title', 'description'], drop_first=True)

# # Step 3: Handle date columns
# # If not using, drop them
# train_data.drop(columns=['original_reg_date', 'reg_date'], inplace=True)

# # If you need to extract features from dates, you can do it like this:
# # df['original_reg_year'] = df['original_reg_date'].dt.year
# # df['original_reg_month'] = df['original_reg_date'].dt.month

# train_data.drop(columns=['indicative_price','listing_id','opc_scheme','eco_category','accessories'], inplace=True)

# # Verify the updated DataFrame types
# print(train_data.dtypes)


In [ ]:
# from datetime import datetime
# import pandas as pd

# # Step 1: Remove duplicate rows based on 'listing_id'
# train_data = train_data.drop_duplicates(subset=['listing_id'])

# # Step 2: Remove rows with Manufacture Year greater than the current year
# current_year = datetime.now().year
# train_data = train_data[train_data['manufactured'] <= current_year]

# # Step 3: Remove rows with Registration Date greater than the current date
# train_data = train_data[pd.to_datetime(train_data['reg_date'], format='mixed', dayfirst=True) <= datetime.now()]

# # Step 4: Drop the 'listing_id' column as it's an artificial column
# train_data = train_data.drop(columns=['listing_id'])

# # Step 5: Remove columns with only one unique value
# same_value_columns = train_data.columns[train_data.nunique() == 1]
# train_data = train_data.drop(columns=same_value_columns)

# # Step 6: Remove free-text columns: 'title', 'description', 'features', 'accessories'
# train_data = train_data.drop(columns=['title', 'description', 'features', 'accessories'])

# # Step 7: Remove columns with more than 50% missing values
# na_columns = train_data.columns[train_data.isnull().mean() > 0.5]
# train_data = train_data.drop(columns=na_columns)

# # Step 8: Calculate 'coe_days' as the difference in days between today and 'reg_date'
# train_data['coe_days'] = (datetime.now() - pd.to_datetime(train_data['reg_date'], format='mixed', dayfirst=True)).dt.days


# # Step 9: Drop the 'reg_date' column after calculating 'coe_days'
# train_data = train_data.drop(columns=['reg_date'])

# # Step 10: One-hot encode categorical columns
# train_data = pd.get_dummies(train_data, drop_first=True)

# # Step 11: Replace remaining missing values with the mean for numeric columns
# for column in train_data.select_dtypes(include=['float64', 'int64']).columns:
#     train_data[column] = train_data[column].fillna(train_data[column].mean())

# # Final cleaned train_data
# train_data.head()
